In [3]:
import numpy as np
import os

# Path ke folder awal (ubah sesuai lokasi di PC kamu)
base_path = "data/"

# Loop untuk setiap fungsi
for i in range(1, 9):
    folder = os.path.join(base_path, f"function_{i}")
    input_file = os.path.join(folder, "initial_inputs.npy")

    # Load data awal untuk tahu dimensinya
    inputs = np.load(input_file)
    dim = inputs.shape[1]  # jumlah kolom = dimensi fungsi

    # Generate random query point [0,1] dengan 6 desimal
    query = np.random.uniform(0, 1, dim)
    query_str = "-".join([f"{x:.6f}" for x in query])

    print(f"Function {i} query: {query_str}")

Function 1 query: 0.812440-0.770492
Function 2 query: 0.386715-0.453987
Function 3 query: 0.951842-0.510049-0.979283
Function 4 query: 0.361902-0.889972-0.126506-0.211285
Function 5 query: 0.890611-0.513240-0.628448-0.974823
Function 6 query: 0.152101-0.519852-0.146528-0.453370-0.142324
Function 7 query: 0.404027-0.530653-0.110653-0.667289-0.332318-0.927179
Function 8 query: 0.541428-0.954806-0.108790-0.410599-0.428328-0.793046-0.111207-0.861138


In [4]:
import numpy as np
import os

# Path ke folder awal (ubah sesuai lokasi di PC kamu)
base_path = "data/"

# Loop untuk setiap fungsi
for i in range(1, 9):
    folder = os.path.join(base_path, f"function_{i}")
    input_file = os.path.join(folder, "initial_inputs.npy")
    output_file = os.path.join(folder, "initial_outputs.npy")

    # Load data awal
    X_init = np.load(input_file)
    y_init = np.load(output_file)
    dim = X_init.shape[1]  # jumlah dimensi input

    # Info ringkas dataset
    print(f"\nFunction {i}:")
    print(f"  Dimensi input: {dim}")
    print(f"  Jumlah titik awal: {len(X_init)}")
    print(f"  Output min={y_init.min():.4f}, max={y_init.max():.4f}, mean={y_init.mean():.4f}")

    # (Contoh) Generate 1 random query point [0,1] dengan 6 desimal
    query = np.random.uniform(0, 1, dim)
    query_str = "-".join([f"{x:.6f}" for x in query])
    print(f"  Query kandidat: {query_str}")


Function 1:
  Dimensi input: 2
  Jumlah titik awal: 10
  Output min=-0.0036, max=0.0000, mean=-0.0004
  Query kandidat: 0.137095-0.562317

Function 2:
  Dimensi input: 2
  Jumlah titik awal: 10
  Output min=-0.0656, max=0.6112, mean=0.2307
  Query kandidat: 0.739027-0.719252

Function 3:
  Dimensi input: 3
  Jumlah titik awal: 15
  Output min=-0.3989, max=-0.0348, mean=-0.1072
  Query kandidat: 0.211421-0.640837-0.789505

Function 4:
  Dimensi input: 4
  Jumlah titik awal: 30
  Output min=-32.6257, max=-4.0255, mean=-17.2386
  Query kandidat: 0.213462-0.298057-0.264448-0.360563

Function 5:
  Dimensi input: 4
  Jumlah titik awal: 20
  Output min=0.1129, max=1088.8596, mean=151.2719
  Query kandidat: 0.311614-0.229620-0.127388-0.980849

Function 6:
  Dimensi input: 5
  Jumlah titik awal: 20
  Output min=-2.5712, max=-0.7143, mean=-1.4954
  Query kandidat: 0.505867-0.192728-0.012810-0.695750-0.238208

Function 7:
  Dimensi input: 6
  Jumlah titik awal: 30
  Output min=0.0027, max=1.3650

In [2]:
import numpy as np
import os
import ast
from sklearn.linear_model import LinearRegression

# === 1. KONFIGURASI ===
base_path = "data/"
inputs_txt = "inputs.txt"
outputs_txt = "outputs.txt"

# === 2. LOAD DATA MINGGU LALU DARI EMAIL ===
# File ini berformat literal Python array, bukan CSV → gunakan ast.literal_eval
with open(inputs_txt, "r") as f:
    text = f.read().strip()
new_inputs = ast.literal_eval(text)   # ini jadi list of numpy arrays

# pastikan setiap elemen adalah np.array
new_inputs = [np.array(x) for x in new_inputs]

# Outputs (lebih sederhana)
with open(outputs_txt, "r") as f:
    text_out = f.read().strip()
new_outputs = np.array(ast.literal_eval(text_out))

print(f"Loaded {len(new_inputs)} input sets, {len(new_outputs)} outputs.")

# === 3. PROSES SETIAP FUNCTION ===
for i in range(1, 9):
    folder = os.path.join(base_path, f"function_{i}")
    input_file = os.path.join(folder, "initial_inputs.npy")
    output_file = os.path.join(folder, "initial_outputs.npy")

    # Load data awal
    X_init = np.load(input_file)
    y_init = np.load(output_file)

    # Gabungkan dengan hasil minggu lalu
    X_combined = np.vstack([X_init, new_inputs[i - 1]])  # tambahkan 1 baris
    y_combined = np.append(y_init, new_outputs[i - 1])

    # Simpan versi updated
    np.save(os.path.join(folder, "updated_inputs.npy"), X_combined)
    np.save(os.path.join(folder, "updated_outputs.npy"), y_combined)

    dim = X_combined.shape[1]
    print(f"\n=== Function {i} ({dim}D) ===")
    print(f"Data size: {X_combined.shape}, Output range: [{y_combined.min():.4f}, {y_combined.max():.4f}]")

    # === 4. TRAIN LINEAR REGRESSION ===
    model = LinearRegression()
    model.fit(X_combined, y_combined)

    # === 5. GENERATE CANDIDATE POINTS & PREDIKSI ===
    candidates = np.random.uniform(0, 1, (1000, dim))
    preds = model.predict(candidates)

    # Ambil titik dengan prediksi tertinggi (eksploitasi)
    best_idx = np.argmax(preds)
    best_query = candidates[best_idx]

    # === 6. CETAK HASIL DALAM FORMAT PORTAL ===
    query_str = "-".join([f"{x:.6f}" for x in best_query])
    print(f"Best predicted output: {preds[best_idx]:.4f}")
    print(f"Query to submit: {query_str}")

ValueError: malformed node or string on line 1: <ast.Call object at 0x0000027573BDBD50>